# Notebook prática

## Descrição do dataset
<a id='description'></a>
Fonte:[
default of credit card clients Data Set](https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients#)

__Dicionário__

* **ID**  - Código de identificação do cliente
* **LIMIT_BAL** - Montante do crédito concedido (em dólar): inclui o crédito individual ao consumidor e o crédito à família (complementar).
* **SEX** - Sexo (1 = Masculino, 2 = Feminino)     
* **EDUCATION** - Nível de escolaridade (1 = pós-graduação; 2 = universidade; 3 = ensino médio; 4 = outros)
* **MARRIAGE** - Estado civíl (1 = casado; 2 = solteiro; 3 = outros)
* **AGE** - idade em anos     
* **PAY_0** - Status de pagamento em Setembro de 2005 (-1 = pagamento devidamente; 1 = atraso no pagamento por um mês; 2 = atraso no pagamento por dois meses; . . .; 8 = atraso no pagamento por oito meses; 9 = atraso no pagamento por nove meses e acima).  
* **PAY_2** - Status de pagamento em Agosto de 2005   
* **PAY_3** - Status de pagamento em Julho de 2005   
* **PAY_4** - Status de pagamento em Junho de 2005   
* **PAY_5** - Status de pagamento em Maio de 2005   
* **PAY_6** - Status de pagamento em Abril de 2005      
* **BILL_AMT1** - Valor do extrato da fatura (em dólar) em Setembro de 2005
* **BILL_AMT2** - Valor do extrato da fatura (em dólar) em Agosto de 2005
* **BILL_AMT3** - Valor do extrato da fatura (em dólar) em Julho de 2005
* **BILL_AMT4** - Valor do extrato da fatura (em dólar) em Junho de 2005
* **BILL_AMT5** - Valor do extrato da fatura (em dólar) em Maio de 2005
* **BILL_AMT6** - Valor do extrato da fatura (em dólar) em Abril de 2005
* **PAY_AMT1** - Valor do pagamento anterior (em dólar)  em Setembro de 2005
* **PAY_AMT2** - Valor do pagamento anterior (em dólar)  em Agosto de 2005
* **PAY_AMT3** - Valor do pagamento anterior (em dólar)  em Julho de 2005
* **PAY_AMT4** - Valor do pagamento anterior (em dólar)  em Junho de 2005
* **PAY_AMT5** - Valor do pagamento anterior (em dólar)  em Maio de 2005
* **PAY_AMT6** - Valor do pagamento anterior (em dólar)  em Abril de 2005 
* **default payment next month** - (0=Não | 1=Sim) Se o cliente é default no mês seguinte (Outubro)

## Imports

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

pd.set_option('display.max_columns', None)

customPal = sns.diverging_palette(100, 0, 74, 39, 19, 2, center='dark')
cmap = sns.diverging_palette(
    0, 100, 74, 39, 19, 25, center='light', as_cmap=True)
sns.set_theme(style="white", palette='RdYlGn_r', font_scale=1.25)

## Funções

## Sumário

1. [Descrição do dataset](#description)
2. [Ingestão dos dados](#ingestion)
3. [Análise expoloratória inicial](#eda)
4. [Análise univariada](#uda)
5. [Análise multivariada](#mda)
6. [Análise de inconsistências](#incon)
7. [Feature Engineering](#feateng)


## Ingestão dos dados

<a id='ingestion'></a>

In [ ]:
PATH = 'data/'
FILENAME = 'default of credit card clients.xls'

df = pd.read_excel(os.path.join(PATH, FILENAME), header=[1])
df.head(2)

## Análise exploratória inicial
<a id='eda'></a>

In [ ]:
df.info()

In [ ]:
df.describe()

### Valores ausentes

In [ ]:
df.isna().sum()

### Duplicados

In [ ]:
print(df.duplicated().sum(), 'valores duplicados')

### Variáveis categóricas

In [ ]:
df['SEX'].value_counts()

In [ ]:
df['MARRIAGE'].value_counts()

In [ ]:
df['EDUCATION'].value_counts()

### Variáveis numéricas

In [ ]:
df[['AGE', 'LIMIT_BAL']].describe()

In [ ]:
df[['PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']].describe()

In [ ]:
print('Valores únicos de status de pagamento:', sorted(df['PAY_0'].unique()))

In [ ]:
df[['BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3',
    'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6']].describe()

In [ ]:
df[['PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']].describe()

__Considerações iniciais__
* No dataset não há valores missing ou duplicados
* As variáveis de status de pagamento (``PAY_0``, ..., ``PAY_6``) possuem valores entre -2 e 8. Fazendo uma busca rápida, descobri que -2 significa que não houve utilização do crédito e 0 signifca uso do crédito rotativo [[1]](https://www.researchgate.net/publication/326171439_Default_Payment_Analysis_of_Credit_Card_Clients).
* Valores negativos nas variáveis de extrato da fatura (``BILL_AMT1``, ..., ``BILL_AMT6``) indicam que o cliente pagou acima do valor devido, no entanto, essa hipótese será verificada nas análises posteriores.
* As variáveis ``SEX``e ``AGE``estão dentro dos valores normais. A idade dos clientes varia entre 21 e 79 anos e a variável sexo possui apenas as categorias masculino e feminino.   
* As variáveis ``EDUCATION``e ``MARRIAGE`` apresentam valores fora das faixas especificadas na base, portanto, esses valores serão agrupados entre as categorias conhecidas.   
``EDUCATION`` = 5, 6 e 0 será agrupada na categoria 4   
``MARRIAGE`` = 0 será agrupada na categoria 3

In [ ]:
df.loc[df['EDUCATION'].isin([5, 6, 0]), 'EDUCATION'] = 4
df.loc[df['MARRIAGE'].isin([0]), 'MARRIAGE'] = 3

### Renomeando colunas

In [ ]:
df.rename(columns={'default payment next month': 'y',
                   'PAY_0': 'pgto_set',
                   'PAY_2': 'pgto_ago',
                   'PAY_3': 'pgto_jul',
                   'PAY_4': 'pgto_jun',
                   'PAY_5': 'pgto_mai',
                   'PAY_6': 'pgto_abr',
                   'PAY_AMT1': 'pgto_amt_set',
                   'PAY_AMT2': 'pgto_amt_ago',
                   'PAY_AMT3': 'pgto_amt_jul',
                   'PAY_AMT4': 'pgto_amt_jun',
                   'PAY_AMT5': 'pgto_amt_mai',
                   'PAY_AMT6': 'pgto_amt_abr',
                   'BILL_AMT1': 'fatura_set',
                   'BILL_AMT2': 'fatura_ago',
                   'BILL_AMT3': 'fatura_jul',
                   'BILL_AMT4': 'fatura_jun',
                   'BILL_AMT5': 'fatura_mai',
                   'BILL_AMT6': 'fatura_abr',
                   'SEX': 'genero',
                   'EDUCATION': 'educacao',
                   'LIMIT_BAL': 'limite',
                   'AGE': 'idade',
                   'MARRIAGE': 'estado_civil'}, inplace=True)

## Análise univariada

Aqui as variáveis categóricas serão renomeadas para facilitar a interpretação dos gráficos.

In [ ]:
df['genero'] = df['genero'].replace({1: 'masculino', 2: 'feminino'})
df['educacao'] = df['educacao'].replace(
    {1: 'pós-graduação', 2: 'superior', 3: 'ensino médio', 4: 'outros'})
df['estado_civil'] = df['estado_civil'].replace(
    {1: 'casado', 2: 'solteiro', 3: 'outros'})

In [ ]:
df.head()

In [ ]:
df_pgto = df[['pgto_set', 'pgto_ago', 'pgto_jul',
              'pgto_jun', 'pgto_mai', 'pgto_abr']]
df_fatura = df[['fatura_set', 'fatura_ago', 'fatura_jul',
                'fatura_jun', 'fatura_mai', 'fatura_abr']]
df_pgto_amt = df[['pgto_amt_set', 'pgto_amt_ago', 'pgto_amt_jul',
                  'pgto_amt_jun', 'pgto_amt_mai', 'pgto_amt_abr']]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
def plot_bars(df, features, n_rows, n_cols, title, figsize):

    fig = plt.figure(figsize=figsize)
    for i, feat in enumerate(features):
        ax = fig.add_subplot(n_rows, n_cols, i+1)
        sns.countplot(data=df, x=feat, ax=ax)

    fig.suptitle(title)
    fig.show()

In [ ]:
plot_bars(df, features=['y', 'genero', 'estado_civil', 'educacao'], n_rows=2, n_cols=2,
          title='Variáveis categóricas', figsize=(18, 12))

In [ ]:
def plot_hists(df, features, n_rows, n_cols, title, figsize):

    fig = plt.figure(figsize=figsize)
    for i, feat in enumerate(features):
        ax = fig.add_subplot(n_rows, n_cols, i+1)
        sns.histplot(df[feat], bins=20, ax=ax, kde=True)

    fig.suptitle(title)
    fig.show()

In [ ]:
plot_hists(df, features=['idade', 'limite'], n_rows=2, n_cols=2,
           title='Distribuição das variáveis numéricas', figsize=(18, 12))

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 6), sharey=True)
sns.boxplot(x="Faturas", y="$ dolar", data=pd.melt(
    df_fatura, var_name='Faturas', value_name='$ dolar'), ax=ax[0])
sns.boxplot(x="Pagamentos", y="$ dolar", data=pd.melt(
    df_pgto_amt, var_name='Pagamentos', value_name='$ dolar'), ax=ax[1])

ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation=90)
ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation=90)
plt.suptitle('Pagamentos e Faturas')
plt.show()

In [ ]:
plt.figure(figsize=(20, 8))
sns.countplot(x="Pagamentos", hue="status",
              data=pd.melt(df_pgto, var_name='Pagamentos', value_name='status'), palette='RdYlGn_r')
plt.title('Status de pagamentos')
plt.show()

__Considerações da análise univarida__
* A base possui 30.000 observações e cada observação corresponde a um cliente diferente sendo 23.364 (77,9%) de clientes que permaneceram em default e 6.636 (22,1%) de clientes que conseguiram honrar seus compromissos de crédito no mês de outubro, ou seja, deixaram de ser inadimplentes;   
* Quanto à distribuição de gênero, predomina o gênero feminino com (60,4%);
* Quanto à distribuição do nível de escolaridade da base, podemos confirmar que a grande maioria está presente nas faixas mais altas. 84% (24.615) da base tem grau de escolaridade acima do nível _high school_ , equivalente ao ensino médio brasileiro. Apenas 1,5% (468) tem um nível de escolaridade desconhecido ou abaixo do ensino médio.
* O estado civíl dos clientes encontra-se balanceado, sendo 15.964 clientes solteiros, 13.659 clientes casados e 377 clientes que provavelmente são divorciados ou viúvos, marcados na categoria _others_ ;
* A idade dos clientes está entre 21 e 79 anos, no entanto, 75% da base tem abaixo de 41 anos.
* O limite dos clientes varia entre 10.000 e 1.000.000 dólares, no entanto, apenas 25% dos clientes têm limite acima de 240.000 dólares e apenas 1 cliente na base possui o limite máximo.
* A maioria da base costuma utilizar predominantemente o crédito rotativo. O crédito rotativo, correspondente ao status de pagamento igual a zero e equivale em média a 52% do status de pagamento da base. Esse status aparece quando o cliente não é capaz de pagar o valor completo da fatura.
* A média dos valores das faturas vão aumentando ao longo do tempo. Em abril, as faturas eram em média no valor de 38.871 dólares, já no mês de setembro esse número foi para 51.223. Os valores mais altos foram registrados no mês de Julho, chegando até 1.6 milhão de dólares. Uma hipótese que justifique esse fenômeno é que boa parte dos clientes costumam usar a modalidade do crédito rotativo a medida que não são capazes de pagar o valor total da fatura, sendo assim os juros acrescidos tende a aumentar o valor das faturas dos meses posteriores.
* Foram identificadas muitas faturas com valores negativos, a hipótese inicial é de que o cliente tenha pago acima do valor devido.
* A média dos valores dos pagamentos oscila entre 4.799 dólares, registrado no mês de maio, até 5.921 dólares, referente ao pagamento mês de agosto.


## Análise Multivariada
<a id='mda'></a>

In [ ]:
num_feat_corr = df.select_dtypes(exclude='object').iloc[:, 1:].corr()
mask = np.triu(np.ones_like(num_feat_corr, dtype=bool))
plt.figure(figsize=(22, 10))
sns.heatmap(num_feat_corr, mask=mask, fmt='.2f', cmap=cmap, annot=True)
plt.title('Correlação de Pearson entre as variáveis numéricas')
plt.show()

In [ ]:
plt.figure(figsize=(20, 4))
corrmat = df.select_dtypes(
    exclude='object').iloc[:, 1:].corr(method='spearman')
sns.heatmap([corrmat['y']], xticklabels=corrmat.index,
            annot=True, fmt='.2f', annot_kws={'size': 14},
            cbar=False, center=0, cmap=cmap)
plt.title('Correlação pelo método de Spearman')

plt.tight_layout()
plt.show()

__Chi-square__

$H_0$ : Variável e target são independentes (Não associadas)   
$H_1$: Variável e target não são independentes (São associadas) 

Resultado:

p-valor $\leq \alpha$ : Rejeitar a hipótetse nula ($H_0$), há dependencia entre a varável e o target.   
p-valor > $\alpha$: Não há evidencias suficientes para rejeitar $H_0$, variável e target independentes.

In [ ]:
from scipy.stats import chi2_contingency


def chisquare(data, target, alpha=0.05):

    features_cat = data.select_dtypes(exclude=['float', 'int']).columns
    for feat in features_cat:
        data_crosstab = pd.crosstab(data[feat], data[target], margins=False)
        _, p, _, _ = chi2_contingency(data_crosstab)

        if p <= alpha:
            print(f'Há dependencia entre {feat} e {target}. P-valor: {p}')
        else:
            print(f'Não há dependencia entre {feat} e {target}. P-valor: {p}')

In [ ]:
chisquare(df, target='y')

**Análises mistas**

In [ ]:
def corr_2_cols(df, col1, col2):
    res = df.groupby([col1, col2]).size().unstack()
    res['frequencia(%)'] = (res[res.columns[1]] /
                            (res[res.columns[0]] + res[res.columns[1]]))
    return res

In [ ]:
# Criando faixas para variáveis numéricas
df['limit_bins'] = pd.qcut(df['limite'], q=10).astype('str')
df['idade_bins'] = pd.qcut(df['idade'], q=10).astype('str')
df['pgto_amt_set_bins'] = pd.qcut(
    df['pgto_amt_set'], q=10, duplicates='drop').astype('str')

# Bad rate por variável
idade_bins = corr_2_cols(df, df['idade_bins'], df['y'])
limit_bins = corr_2_cols(df, df['limit_bins'], df['y'])
pgto_amt_set_bins = corr_2_cols(df, df['pgto_amt_set_bins'], df['y'])
genero = corr_2_cols(df, df['genero'], df['y']).sort_values('frequencia(%)')
educacao = corr_2_cols(df, df['educacao'], df['y']
                       ).sort_values('frequencia(%)')
estado_civil = corr_2_cols(
    df, df['estado_civil'], df['y']).sort_values('frequencia(%)')

In [ ]:
fig, ax = plt.subplots(5, 3, figsize=(32, 36))
sns.countplot(x='genero', hue='y', data=df, ax=ax[0, 0], palette=customPal)
sns.countplot(x='educacao', hue='y', data=df, ax=ax[0, 1], palette=customPal)
sns.countplot(x='estado_civil', hue='y', data=df,
              ax=ax[0, 2], palette=customPal)
sns.boxplot(x='educacao', y='limite', hue='y',
            data=df, ax=ax[1, 0], palette=customPal)
sns.boxplot(x='estado_civil', y='limite', hue='y',
            data=df, ax=ax[1, 1], palette=customPal)
sns.boxplot(x='genero', y='limite', hue='y',
            data=df, ax=ax[1, 2], palette=customPal)
sns.scatterplot(y='limite', x='pgto_set',  hue='y',
                data=df, ax=ax[2, 0], palette=customPal)
sns.scatterplot(x='limite', y='idade',  hue='y',
                data=df, ax=ax[2, 1], palette=customPal)
sns.kdeplot(data=df[df['y'] == 0]['idade'], shade=True,
            label='y = 0', color='g', ax=ax[2, 2])
sns.kdeplot(data=df[df['y'] == 1]['idade'], shade=True,
            label='y = 1', color='r', ax=ax[2, 2])


sns.barplot(data=genero, x=genero.index,
            y='frequencia(%)', ax=ax[3, 0], palette='Reds')
sns.barplot(data=educacao, x=educacao.index,
            y='frequencia(%)', ax=ax[3, 1], palette='Reds')
sns.barplot(data=estado_civil, x=estado_civil.index,
            y='frequencia(%)', ax=ax[3, 2], palette='Reds')
sns.barplot(data=idade_bins, x=idade_bins.index,
            y='frequencia(%)', palette='Reds_r', ax=ax[4, 0])
sns.barplot(data=limit_bins, x=limit_bins.index,
            y='frequencia(%)', palette='Reds', ax=ax[4, 1])
sns.barplot(data=pgto_amt_set_bins, x=pgto_amt_set_bins.index,
            y='frequencia(%)', palette='Reds_r', ax=ax[4, 2])


# nomeando eixos do kdeplot
ax[3, 2].set_xlabel('idade')
ax[3, 2].set_ylabel('frequência')

# rotacionando legenda do eixo x
ax[4, 0].set_xticklabels(ax[4, 0].get_xticklabels(), rotation=90)
ax[4, 1].set_xticklabels(ax[4, 1].get_xticklabels(), rotation=90)
ax[4, 2].set_xticklabels(ax[4, 2].get_xticklabels(), rotation=90)

plt.show()

In [ ]:
plt.figure(figsize=(20, 8))
sns.boxplot(x='idade', y='limite', hue='y', data=df, palette=customPal)
plt.title('Variação do limite pela idade')
plt.show()

__Considerações da análise multivariada__

* As relações entre as colunas ``fatura`` e ``pgto`` se dão da seguinte forma:   
A fatura do mês Abril é paga no mês de Maio, sendo assim, o **BILL_AMT_6** deve ser pago no **PAY_AMT_5**, o **BILL_AMT_5** deve ser pago no **PAY_AMT_4** e assim por diante.
* A correlação de Pearson entre colunas ``fatura`` são altamente correlacionada, o mesmo acontece entre as variáveis de status de pagamento, ``pgto``, essas variaveis e devem ser agrupadas se forem utilizadas em modelos lineares como a regressão logistica
* A correlação de Spearman mede a relação monotônica entre variáveis e é mais indicada na correlação entre variaveis categoricas e númericas. Foi verificado que a ``pgto_set`` possui a maior correlação o target.
* O teste de chi-square foi utilizado para medir a dependência entre as variaveis categóricas com o target. Os p-valores abaixo de 0,05 reforçam que há dependência entre as variáveis categóricas e o default.
* 60% da base são mulheres e dessas, 20% são clientes default, já os homens correspondem a cerca de 40% da base e 24% dos clientes default.
* 25% dos clientes default possuem apenas o ensino médio, apenas 19% possuem pós graduação.
* 23% dos clientes default possuem estado civil outros.
* De modo geral, os limites dos clientes default são inferiores aos limites do clientes não default, embora  essa lógica pareça mudar para clientes com idades nas faixas mais altas, não há volumetria suficiente para concluirmos isso.
* O status de pagemento de setembro é uma variavel bastante explicativa, pois os clientes default estão concentrados predominantemente nos status entre 1 e 8.

## Feature Engineering
<a id='feateng'></a>

* **atraso** - variável binária que indica se o cliente está atrasado no mês analisado.
* **saldo** - diferença entre a fatura e o pagamento no mês analisado
* **despesa** - as despesas são a fatura do mês atual somado aos pagamentos das faturas dos meses anteriores e normalizado pelo limite.
* **limit_rate** - o percentual do limite que foi consumido no mês

### Atraso

As variaveis de status de pagamento possuem um forte poder preditivo quando comparado com as demais variaveis presentes no dataset, porém, devido às inconsistências encontradas nessas variáveis, foi visto a necessidade da criação de variaveis mais interpretáveis e nesse sentido foram criadas as colunas que indicam o atraso menasal. Esse grupo de variáveis diz se o cliente está em atraso ou não com base nas colunas de status de pagamento e em regras de negócio.

Os valores dos staus de pagamento -1 e -2 indicam, respectivamente, se o cliente está em dia com o pagamento ou se não está utilizando o crédito. Os valores do status de pagamento positivo, indicam a quantidade de meses em atraso. No entanto, o valor ZERO indica que o cliente encontra-se utilizando o crédito rotativo. Será verificado se esse  status de pagamento tem maior correlação com o default quando classificado como atraso.

Assumindo que crédito rotativo seja uma caracteristica de clientes predominantemente em dia, foi criada a dummy ``atraso`` :

\begin{equation*}
atraso_i = \left\{\begin{matrix}
0, ~pgto_i \leq 0\\ 
1, ~pgto_i > 0
\end{matrix}\right.
\end{equation*}

, em que _i_ representa o mês no histórico dos clientes

In [ ]:
df['atraso_set'] = np.where(df['pgto_set'] > 0, 1, 0)
df['atraso_ago'] = np.where(df['pgto_ago'] > 0, 1, 0)
df['atraso_jul'] = np.where(df['pgto_jul'] > 0, 1, 0)
df['atraso_jun'] = np.where(df['pgto_jun'] > 0, 1, 0)
df['atraso_mai'] = np.where(df['pgto_mai'] > 0, 1, 0)
df['atraso_abr'] = np.where(df['pgto_abr'] > 0, 1, 0)

A variavel ``atraso_set`` indica se o cliente está em atraso no mês de setembro, sua correlação com o target já é maior do que todas as demais variáveis presentes no dataset, indicando que esta é uma variável promissora que provavlemente poderá melhorar a performance dos modelos, além do fato de ser mais interpretável que as variáveis de status de pagamento.

Levando em consideração as análises prévias das inconsistências nas variáveis de status de pagamento, serão considerados clientes sem atraso aqueles que possuem a fatura negativa.

In [ ]:
df['atraso_set'] = np.where(df['pgto_set'] > 0, 1, 0)
df.loc[df['fatura_set'] < 0, 'atraso_set'] = 0

df['atraso_ago'] = np.where(df['pgto_ago'] > 0, 1, 0)
df.loc[df['fatura_ago'] < 0, 'atraso_ago'] = 0

df['atraso_jul'] = np.where(df['pgto_jul'] > 0, 1, 0)
df.loc[df['fatura_jul'] < 0, 'atraso_jul'] = 0

df['atraso_jun'] = np.where(df['pgto_jun'] > 0, 1, 0)
df.loc[df['fatura_jun'] < 0, 'atraso_jun'] = 0

df['atraso_mai'] = np.where(df['pgto_mai'] > 0, 1, 0)
df.loc[df['fatura_mai'] < 0, 'atraso_mai'] = 0

df['atraso_abr'] = np.where(df['pgto_abr'] > 0, 1, 0)
df.loc[df['fatura_abr'] < 0, 'atraso_abr'] = 0

### Saldo

Esse grupo de variáveis numéricas mede a diferença entre a fatura e o pagamento mensal normalizado pelo limite.

\begin{equation*}
saldo_i = \frac{fatura_i - pagamento_{i+1}}{limite} 
\end{equation*}

, em que $i$ representa o mês atual e $i+1$ o mês futuro em que é realizado o pagamento do mês anterior, portanto, o pagamento da $fatura_i$.

Por exemplo, o fechamento da fatura de abril é realizado no final do mês e seu pagamento é realizado no mês de maio, logo, o saldo de abril é exatamente o quanto da fatura de abril foi paga pelo cliente.
Como o pagamento da fatura de setembro só é realizado em outubro, as variáveis de saldo são do mês de abril ao mês de agosto.

In [ ]:
df['saldo_amt_ago'] = (df['fatura_ago'] - df['pgto_amt_set'])/df['limite']
df['saldo_amt_jul'] = (df['fatura_jul'] - df['pgto_amt_ago'])/df['limite']
df['saldo_amt_jun'] = (df['fatura_jun'] - df['pgto_amt_jul'])/df['limite']
df['saldo_amt_mai'] = (df['fatura_mai'] - df['pgto_amt_jun'])/df['limite']
df['saldo_amt_abr'] = (df['fatura_abr'] - df['pgto_amt_mai'])/df['limite']

### Despesas

As despesas são a fatura do mês atual somado aos pagamentos das faturas dos meses anteriores e normalizado pelo limite.

\begin{equation*}
despesa_n = \frac{fatura_{n} + \sum_{i}^{n} (fatura_{i} - pagamento_{i+1})}{limite} 
\end{equation*}

In [ ]:
df['despesa_abr'] = (df_fatura.iloc[:, -2:].sum(axis=1) -
                     df_pgto_amt['pgto_amt_mai']) / df['limite']
df['despesa_mai'] = (df_fatura.iloc[:, -3:].sum(axis=1) -
                     df_pgto_amt.iloc[:, -2:].sum(axis=1)) / df['limite']
df['despesa_jun'] = (df_fatura.iloc[:, -4:].sum(axis=1) -
                     df_pgto_amt.iloc[:, -3:].sum(axis=1)) / df['limite']
df['despesa_jul'] = (df_fatura.iloc[:, -5:].sum(axis=1) -
                     df_pgto_amt.iloc[:, -4:].sum(axis=1)) / df['limite']
df['despesa_ago'] = (df_fatura.iloc[:, -6:].sum(axis=1) -
                     df_pgto_amt.iloc[:, -5:].sum(axis=1)) / df['limite']

### Porcentagem do limite utilizado

A porcentagem do limite utilizado é basicamente o quanto do limite o cliente consome por mês, intuitivamente, clientes que tem uma utilização de crédito próxima ao valor do seu limite tende a ser mais arriscado. Clientes com faturas negativas possuem um limit_rate negativo, no entanto, o valor mínimo de limit_rate foi truncado para 0, uma vez que não tem muito sentido prático afirmar que o clientes possui uma taxa de consumo de limite negativa.

\begin{equation*}
limit ~rate_i = 1 -\frac{limite - fatura_{i}} {limite}
\end{equation*}

In [ ]:
df['limit_rate_set'] = 1-((df['limite'] - df['fatura_set']) / df['limite'])
df['limit_rate_ago'] = 1-((df['limite'] - df['fatura_ago']) / df['limite'])
df['limit_rate_jul'] = 1-((df['limite'] - df['fatura_jul']) / df['limite'])
df['limit_rate_jun'] = 1-((df['limite'] - df['fatura_jun']) / df['limite'])
df['limit_rate_mai'] = 1-((df['limite'] - df['fatura_mai']) / df['limite'])
df['limit_rate_abr'] = 1-((df['limite'] - df['fatura_abr']) / df['limite'])

df.loc[(df['limit_rate_set'] < 0, 'limit_rate_set')] = 0
df.loc[(df['limit_rate_ago'] < 0, 'limit_rate_ago')] = 0
df.loc[(df['limit_rate_jul'] < 0, 'limit_rate_jul')] = 0
df.loc[(df['limit_rate_jun'] < 0, 'limit_rate_jun')] = 0
df.loc[(df['limit_rate_mai'] < 0, 'limit_rate_mai')] = 0
df.loc[(df['limit_rate_abr'] < 0, 'limit_rate_abr')] = 0

## Correlação entre as novas variáveis

In [ ]:
num_feat_corr = df.filter(regex='limit_rate|despesa|saldo|atraso').corr()
mask = np.triu(np.ones_like(num_feat_corr, dtype=bool))
plt.figure(figsize=(22, 10))
sns.heatmap(num_feat_corr, mask=mask, fmt='.2f', cmap=cmap, annot=True)
plt.title('Correlação de Pearson entre as variáveis numéricas')
plt.show()

## Salvando novo dataframe

In [ ]:
df['genero'] = df['genero'].replace({'masculino': 1, 'feminino': 2})
df['educacao'] = df['educacao'].replace(
    {'pós-graduação': 1, 'superior': 2,  'ensino médio': 3, 'outros': 4})
df['estado_civil'] = df['estado_civil'].replace(
    {'casado': 1, 'solteiro': 2, 'outros': 3})

df = df.drop(['limit_bins', 'idade_bins', 'pgto_amt_set_bins'], axis=1)

In [ ]:
df.to_csv('data/data_train_preproc.csv', index=False)

## Conclusão

Com base na análise detalhada do dataset de clientes de cartão de crédito, as seguintes recomendações podem ser feitas para a área de negócio:

1. **Gestão de Risco de Crédito**:
   - Priorizar a análise do status de pagamento, especialmente a variável `atraso_set`, para identificar clientes com maior propensão ao default.
   - Implementar políticas de crédito mais rigorosas para clientes com limites mais baixos e alta utilização do crédito rotativo.

2. **Melhoria na Previsão de Default**:
   - Desenvolver modelos preditivos utilizando variáveis como saldo, despesas e limit rate, que demonstraram ser altamente correlacionadas com o default.
   - Considerar a criação de variáveis derivadas mais interpretáveis para melhorar a interpretação e a precisão dos modelos.

3. **Perfil de Clientes**:
   - Identificar perfis de clientes com maior risco de default, como aqueles com níveis mais baixos de escolaridade ou estados civis menos estáveis.
   - Personalizar estratégias de comunicação e oferta para diferentes segmentos de clientes, com base nas características demográficas e comportamentais.

4. **Melhoria na Qualidade dos Dados**:
   - Tratar as inconsistências identificadas nas variáveis de status de pagamento e faturas negativas para melhorar a qualidade dos dados de entrada nos modelos.

5. **Monitoramento Contínuo**:
   - Implementar um sistema de monitoramento contínuo dos clientes para identificar mudanças nos padrões de comportamento e de risco.
   - Atualizar regularmente os modelos preditivos com novos dados e ajustar as estratégias conforme necessário.

Essas recomendações visam melhorar a gestão de risco de crédito, aumentar a precisão na previsão de default e proporcionar uma abordagem mais personalizada para o relacionamento com os clientes. Implementar essas estratégias pode resultar em uma redução significativa nos índices de inadimplência e em uma gestão mais eficiente da carteira de clientes de cartão de crédito.